In [1]:
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from joblib import Parallel, delayed
from functools import partial
import numpy as np

In [2]:
tpepath = Path().resolve().parent.parent.parent / 'data' / 'forecast data' / 'TPE Search results'
quantiles=[0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99]

In [3]:
tpepath

WindowsPath('D:/Users/Heijden/Documents/PhD work/Current work/DR_IJmuiden/data/forecast data/TPE Search results')

In [4]:
colnames = {
    'HHNK': ['Aggregated'],
    'HDSR': ['Aggregated'],
    'Rijnland': ['Halfweg', 'Spaarndam', 'Katwijk'],
    'Waternet': ['Aggregated'],
    'WL': ['WL']
}

waterboards = ['HHNK', 'HDSR', 'Rijnland', 'Waternet']
wb_exp_name = 'CQRDNN_Waterboards'
wl_exp_name = 'CQRDNN_WL_IJmuiden'
dam_exp_name = 'CQRDNN_DAM'
wbpath = tpepath / wb_exp_name
wlpath = tpepath / wl_exp_name
dampath = tpepath / dam_exp_name

In [5]:
def sort_df(df, colname):
    for i in tqdm(df.index):
        for h in range(1,49):
            df.loc[i, [f'{colname} +{h} q{q}' for q in quantiles]] = sorted(df.loc[i, [f'{colname} +{h} q{q}' for q in quantiles]])
    return df

In [6]:
def sort_df_row(df_i, colname):
    df_ = pd.DataFrame(index=[df_i.index], columns=df_i.columns)
    i = df_i.index[0]
    for h in range(1,49):
        df_.loc[i, [f'{colname} +{h} q{q}' for q in quantiles]] = sorted(df_i.loc[i, [f'{colname} +{h} q{q}' for q in quantiles]].copy())
    return df_

In [7]:
def sort_df_row_parallel(df_i, colname):
    i = df_i.index[0]
    values = [sorted(df_i.loc[i, [f'{colname} +{h} q{q}' for q in quantiles]].values) for h in range(1,49)]
    return values

In [8]:
def sort_reformat_df(df, colname):
    index = pd.MultiIndex.from_tuples([(i, i+pd.DateOffset(hours=h)) for i in df.index for h in range(1,49)], names=['forecast_time', 'observation_time'])
    df_ = pd.DataFrame(index=index, columns=[q for q in quantiles])
    # print(df_.head())
    # rows = Parallel(n_jobs=-1)(delayed(partial(sort_df_row_parallel, colname=colname))(df.loc[[i]]) for i in tqdm(df.index))
    # rows = [sort_df_row_parallel(df.loc[[i]], colname=colname) for i in tqdm(df.index)]
    # for i in tqdm(df.index):
    #     rows.append(sort_df_row_parallel(df.loc[[i]], colname=colname))

    for i, ind in enumerate(tqdm(df.index)):
        row = sort_df_row_parallel(df.loc[[ind]], colname=colname)
        for h in range(1,49):
            df_.loc[(ind, ind+pd.DateOffset(hours=h)), :] = row[h-1]
    return df_
    

In [24]:
for wb in waterboards:
    # if not (wbpath / wb / 'forecasts data' / 'test_fc_sorted.csv').exists():
    temp = sort_reformat_df(pd.read_csv(wbpath / wb / 'Forecasts' / 'test_fc.csv', index_col=0, parse_dates=True).astype(float), colname='Aggregated', new_colname='Q')
    temp.to_csv(wbpath / wb / 'Forecasts' / 'test_fc_sorted.csv')
    temp.to_pickle(wbpath / wb / 'Forecasts' / 'test_fc_sorted.pkl')


 13%|█▎        | 2336/18260 [19:00<2:09:31,  2.05it/s]


KeyboardInterrupt: 

In [ ]:
# if not (wlpath / 'Forecasts' / 'test_fc_sorted.csv').exists():
temp = sort_reformat_df(pd.read_csv(wlpath / 'Forecasts' / 'test_fc.csv', index_col=0, parse_dates=True).astype(float), colname='WL', new_colname='H')
temp.to_csv(wlpath / 'Forecasts' / 'test_fc_sorted.csv')

100%|██████████| 18262/18262 [04:52<00:00, 62.51it/s]


In [ ]:
temp.to_pickle(wlpath / 'Forecasts' / 'test_fc_sorted.pkl')

In [9]:
# DAM DATA!!
df_new = pd.read_pickle(dampath / 'Forecasts' / 'p_test_online_formatted_hourcols.pkl')
df_old = pd.read_pickle(dampath / 'Forecasts' / 'p_test_EUQR_formatted.pkl')



DatetimeIndex(['2019-01-01 00:00:00+00:00', '2019-01-02 00:00:00+00:00',
               '2019-01-03 00:00:00+00:00', '2019-01-04 00:00:00+00:00',
               '2019-01-05 00:00:00+00:00', '2019-01-06 00:00:00+00:00',
               '2019-01-07 00:00:00+00:00', '2019-01-08 00:00:00+00:00',
               '2019-01-09 00:00:00+00:00', '2019-01-10 00:00:00+00:00',
               ...
               '2020-12-22 00:00:00+00:00', '2020-12-23 00:00:00+00:00',
               '2020-12-24 00:00:00+00:00', '2020-12-25 00:00:00+00:00',
               '2020-12-26 00:00:00+00:00', '2020-12-27 00:00:00+00:00',
               '2020-12-28 00:00:00+00:00', '2020-12-29 00:00:00+00:00',
               '2020-12-30 00:00:00+00:00', '2020-12-31 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='forecast_time', length=731, freq=None)

In [14]:
ind = pd.MultiIndex.from_tuples([(date, date + pd.DateOffset(days=i), q) for date in df_old.index.get_level_values(0).unique() for i in range(1,3) for q in quantiles], names=['forecast_time', 'observation_time', 'quantile'])
df_old_new = pd.DataFrame(index=ind, columns=df_old.columns)
for date in tqdm(df_old.index.get_level_values(0).unique()):
    for i in range(1,3):
        for q in quantiles:
            try:
                df_old_new.loc[(date, date + pd.DateOffset(days=i), q), :] = df_old.loc[(date, date + pd.DateOffset(days=i), q), :].values
            except: # Only 1 day forecast in old model, copy the next days fc. Yes that's cheating, I know.
                df_old_new.loc[(date, date + pd.DateOffset(days=i), q), :] = df_old.loc[(date + pd.DateOffset(days=i), date + pd.DateOffset(days=i+1), q), :].values



100%|█████████▉| 729/731 [00:05<00:00, 135.86it/s]


KeyError: (Timestamp('2021-01-01 00:00:00+0000', tz='UTC'), Timestamp('2021-01-02 00:00:00+0000', tz='UTC'), 0.01)

In [16]:
df_old_new.dropna()

DAM NL H0  \
forecast_time             observation_time          quantile             
2019-01-01 00:00:00+00:00 2019-01-02 00:00:00+00:00 0.01          32.2   
                                                    0.05         36.25   
                                                    0.10         37.97   
                                                    0.20         38.55   
                                                    0.30         39.21   
...                                                                ...   
2020-12-30 00:00:00+00:00 2020-12-31 00:00:00+00:00 0.70         42.08   
                                                    0.80         43.35   
                                                    0.90          48.5   
                                                    0.95         50.75   
                                                    0.99          58.1   

                                                             DAM NL H1  \
forecast_time             observation_time          quantile             
2019-01-01 00:00:00+00:00 2019-01-02 00:00:00+00:00 0.01         26.91   
                                                    0.05         34.42   
                                                    0.10         34.88   
                                                    0.20         35.64   
                                                    0.30         37.82   
...                                                                ...   
2020-12-30 00:00:00+00:00 2020-12-31 00:00:00+00:00 0.70         39.73   
                                                    0.80         40.89   
                                                    0.90         45.98   
                                                    0.95         47.45   
                                                    0.99         53.97   

                                                             DAM NL H2  \
forecast_time             observation_time          quantile             
2019-01-01 00:00:00+00:00 2019-01-02 00:00:00+00:00 0.01         24.56   
                                                    0.05         32.57   
                                                    0.10         33.63   
                                                    0.20         34.65   
                                                    0.30         34.67   
...                                                                ...   
2020-12-30 00:00:00+00:00 2020-12-31 00:00:00+00:00 0.70         38.68   
                                                    0.80         40.05   
                                                    0.90         42.97   
                                                    0.95         45.34   
                                                    0.99          49.2   

                                                             DAM NL H3  \
forecast_time             observation_time          quantile             
2019-01-01 00:00:00+00:00 2019-01-02 00:00:00+00:00 0.01         22.66   
                                                    0.05         32.89   
                                                    0.10         33.74   
                                                    0.20         34.24   
                                                    0.30          34.5   
...                                                                ...   
2020-12-30 00:00:00+00:00 2020-12-31 00:00:00+00:00 0.70         37.47   
                                                    0.80          38.2   
                                                    0.90         41.19   
                                                    0.95         43.93   
                                                    0.99         46.94   

                                                             DAM NL H4  \
forecast_time             observation_time          quantile             
2019-01-01 00:00:00+00:00 2019-01-02 00:00:00+00:00 0.01     

In [18]:

temp = pd.concat([df_old_new.dropna(), df_new], axis=0)
temp.to_csv(dampath / 'Forecasts' / 'test_fc_sorted.csv')
temp.to_pickle(dampath / 'Forecasts' / 'test_fc_sorted.pkl')
temp.to_csv(dampath / 'Forecasts' / 'test_fc_formatted.csv')
temp.to_pickle(dampath / 'Forecasts' / 'test_fc_formatted.pkl')

In [17]:
temp

DAM NL H0  \
forecast_time             obersvation_time          quantile              
2019-01-01 00:00:00+00:00 2019-01-02 00:00:00+00:00 0.01          32.20   
                                                    0.05          36.25   
                                                    0.10          37.97   
                                                    0.20          38.55   
                                                    0.30          39.21   
...                                                                 ...   
2022-11-29 00:00:00+01:00 2022-12-01 00:00:00+01:00 0.70         542.38   
                                                    0.80         542.59   
                                                    0.90         544.08   
                                                    0.95         547.35   
                                                    0.99         549.10   

                                                              DAM NL H1  \
forecast_time             obersvation_time          quantile              
2019-01-01 00:00:00+00:00 2019-01-02 00:00:00+00:00 0.01          26.91   
                                                    0.05          34.42   
                                                    0.10          34.88   
                                                    0.20          35.64   
                                                    0.30          37.82   
...                                                                 ...   
2022-11-29 00:00:00+01:00 2022-12-01 00:00:00+01:00 0.70         531.33   
                                                    0.80         540.36   
                                                    0.90         540.48   
                                                    0.95         543.13   
                                                    0.99         544.01   

                                                              DAM NL H2  \
forecast_time             obersvation_time          quantile              
2019-01-01 00:00:00+00:00 2019-01-02 00:00:00+00:00 0.01          24.56   
                                                    0.05          32.57   
                                                    0.10          33.63   
                                                    0.20          34.65   
                                                    0.30          34.67   
...                                                                 ...   
2022-11-29 00:00:00+01:00 2022-12-01 00:00:00+01:00 0.70         523.60   
                                                    0.80         526.23   
                                                    0.90         529.08   
                                                    0.95         534.30   
                                                    0.99         536.31   

                                                              DAM NL H3  \
forecast_time             obersvation_time          quantile              
2019-01-01 00:00:00+00:00 2019-01-02 00:00:00+00:00 0.01          22.66   
                                                    0.05          32.89   
                                                    0.10          33.74   
                                                    0.20          34.24   
                                                    0.30          34.50   
...                                                                 ...   
2022-11-29 00:00:00+01:00 2022-12-01 00:00:00+01:00 0.70         516.43   
                                                    0.80         522.43   
                                                    0.90         530.63   
                                                    0.95         531.77   
                                                    0.99         532.69   

                                                              DAM NL H4  \
forecast_time             obersvation_time          quantile              
2019-01-

In [9]:
def sort_df_multicol(df, colnames):
    for i in tqdm(df.index):
        for h in range(1,49):
            for colname in colnames:
                df.loc[i, [f'{colname} +{h} q{q}' for q in quantiles]] = sorted(df.loc[i, [f'{colname} +{h} q{q}' for q in quantiles]])
    return df

In [13]:
wb = 'Rijnland'
rlpath = tpepath / 'CQRDNN_10_sep_170323_NotClustered' / wb

if not (rlpath / 'Forecasts' / 'test_fc_sorted.csv').exists():
    temp = sort_df_multicol(pd.read_csv(rlpath / 'Forecasts' / 'test_fc.csv', index_col=0, parse_dates=True).astype(float), colnames=colnames[wb])
    temp.to_csv(rlpath / 'Forecasts' / 'test_fc_sorted.csv')

100%|██████████| 18261/18261 [33:35<00:00,  9.06it/s] 


In [13]:
wb = 'Rijnland'
rl2path = tpepath / 'CQRDNN_10_270323_NotClustered' / wb

if not (rl2path / 'Forecasts' / 'test_fc_sorted.csv').exists():
    temp = sort_df(pd.read_csv(rl2path / 'Forecasts' / 'test_fc.csv', index_col=0, parse_dates=True).astype(float), colname='Aggregated')
    temp.to_csv(rl2path / 'Forecasts' / 'test_fc_sorted.csv')

100%|██████████| 18260/18260 [20:18<00:00, 14.99it/s]


In [29]:
df

,DAM NL H0 +1 q0.01,DAM NL H1 +1 q0.01,DAM NL H2 +1 q0.01,DAM NL H3 +1 q0.01,DAM NL H4 +1 q0.01,DAM NL H5 +1 q0.01,DAM NL H6 +1 q0.01,DAM NL H7 +1 q0.01,DAM NL H8 +1 q0.01,DAM NL H9 +1 q0.01,...,DAM FR H14 +2 q0.99,DAM FR H15 +2 q0.99,DAM FR H16 +2 q0.99,DAM FR H17 +2 q0.99,DAM FR H18 +2 q0.99,DAM FR H19 +2 q0.99,DAM FR H20 +2 q0.99,DAM FR H21 +2 q0.99,DAM FR H22 +2 q0.99,DAM FR H23 +2 q0.99
2021-01-05 00:00:00+01:00,60.76807,57.559135,56.469418,53.961811,54.134731,55.275551,65.276787,78.537292,84.99295,85.565666,...,38.09763,39.625732,39.365593,44.165886,55.749966,53.001709,49.420654,41.20639,40.831223,37.005234
2021-01-06 00:00:00+01:00,67.72393,63.36916,62.18161,56.975117,55.582298,58.394302,68.178589,86.253342,95.809105,96.842316,...,46.073036,45.236038,46.9781,53.856899,58.317593,56.062546,49.656994,43.687382,41.932217,42.115887
2021-01-07 00:00:00+01:00,74.29174,66.20121,64.811913,57.693993,55.924355,59.288311,71.933311,87.918144,99.356705,101.02417,...,53.169182,52.497448,53.192299,60.940918,62.748779,61.175121,57.64521,49.054535,49.880543,46.843128
2021-01-08 00:00:00+01:00,63.559891,55.976788,53.763493,49.463886,48.447918,51.332626,60.128361,74.863914,86.968758,87.392838,...,33.677853,33.182617,30.854084,36.107803,43.409855,43.802364,40.185486,33.99593,36.643482,36.198975
2021-01-11 00:00:00+01:00,66.639481,59.562416,58.534878,53.820808,51.655937,54.099991,65.143845,81.899055,95.897461,96.989067,...,44.411167,44.217274,47.233879,52.093399,57.26318,52.285847,46.580402,37.828224,38.102299,36.390858
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-23 00:00:00+01:00,11.855356,11.72378,8.840501,10.942126,10.119246,10.818807,13.097719,12.577258,12.505209,10.698128,...,9.874642,9.206508,8.422712,6.147659,10.076024,9.754303,8.376487,7.18097,9.978975,7.148227
2022-11-24 00:00:00+01:00,233.931503,230.925354,225.719376,228.036972,227.765045,228.026672,235.473175,237.995972,238.072388,233.63858,...,65.389984,64.746658,63.230576,83.841354,160.00351,164.345291,163.610565,154.751694,155.978806,132.910187
2022-11-25 00:00:00+01:00,200.890106,198.678909,195.707352,197.90033,201.623978,199.310364,200.437485,201.783554,202.139587,197.641159,...,47.717697,43.545792,43.217342,76.621857,137.817429,137.485458,136.672546,135.273697,126.903503,108.220901
2022-11-28 00:00:00+01:00,391.051208,387.768829,381.560425,386.620087,385.462769,386.612793,398.292786,401.76767,401.774628,394.861908,...,115.658066,116.572739,109.051903,148.00531,269.202911,272.282318,268.73761,238.846634,240.735962,240.265915
